# 参数的访问

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.3236],
        [0.3332]], grad_fn=<AddmmBackward0>)

In [11]:
# 通过某层(nn.Module)的state_dict()属性来访问权重
print(net[2].state_dict())
# 由于state_dict()返回一个字典，因此可以通过字典的方式来访问
print(net.state_dict()['2.bias'].data)

OrderedDict([('weight', tensor([[ 0.0075,  0.3435,  0.1816,  0.0573,  0.1100, -0.1977,  0.3290, -0.1557]])), ('bias', tensor([0.1735]))])
tensor([0.1735])


In [7]:
# 访问偏置项bias
print(type(net[2].bias))
print(net[2].bias)  # 通过bias属性来访问偏置项
print(net[2].bias.data)  # 通过data来直接访问某个值

# 访问权重weight
print(type(net[2].weight))
print(net[2].weight)  # 通过weight属性来访问权重项
# print(net[2].grad)  # 通过grad属性来访问梯度项，这里没计算梯度时访问会报错
print(net[2].weight.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1735], requires_grad=True)
tensor([0.1735])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([[ 0.0075,  0.3435,  0.1816,  0.0573,  0.1100, -0.1977,  0.3290, -0.1557]],
       requires_grad=True)
tensor([[ 0.0075,  0.3435,  0.1816,  0.0573,  0.1100, -0.1977,  0.3290, -0.1557]])


In [10]:
# 通过named_parameters()函数来一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])  # 网络第一层
print(*[(name, param.shape) for name, param in net.named_parameters()])     # 网络所有参数

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [13]:
# 定义一个嵌套的更复杂的网络结构
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0062],
        [0.0062]], grad_fn=<AddmmBackward0>)

In [14]:
# 通过直接print一个net实例来观察网络结构
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [15]:
# 因为层是分层嵌套的，所以通过嵌套列表索引一样进行访问
# 例如下面访问第一个主要的块，其中第二个子块的第一层的偏置项
rgnet[0][1][0].bias.data

tensor([-0.0744,  0.4349, -0.2997,  0.4357, -0.4793, -0.4873, -0.1084, -0.3985])

# 参数初始化

In [17]:
# apply方法进行内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)  # 有_在最后的函数为原地操作函数(替换m.weight的值而不)
        nn.init.zeros_(m.bias)
        # 还有一些其他方式的初始化函数
        # nn.init.xavier_uniform_(m.weight)  # uniform distribution
        # nn.init.constant_(m.weight, 42)
        
net.apply(init_normal)  # apply函数表示自动对net中的所有module层，应用init_normal函数
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0065, -0.0076,  0.0024,  0.0237]), tensor(0.))

In [18]:
# 也可以有自定义的初始化方式
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5  # 自定义的计算方式

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 8.1403, -0.0000, -0.0000,  0.0000],
        [-0.0000, -7.4179,  7.0769,  7.2525]], grad_fn=<SliceBackward0>)

In [19]:
# 甚至可以更暴力地通过直接访问元素来修改的方式进行参数的设置
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

# 参数绑定

In [20]:
# 我们需要给共享层一个名称，以便可以引用它的参数。
shared = nn.Linear(8, 8)  # 后续每层加入shared层时，类似传引用地会被指引到shared这个层实例
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值。
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
